# 3. Shell és Regex

>Számítógépes nyelvészet, 2018 tavasz

>Simon Eszter, Mittelholcz Iván

>MTA, Nyelvtudományi Intézet

## Tartalom

* [1. Shell](#1.-Shell)
    * [1.1. Alapfogalmak](#1.1.-Alapfogalmak)
    * [1.2. Történelem](#1.2.-Történelem)
    * [1.3. Hogyan működik](#1.3.-Hogy-működik?)
    * [1.4. Fájlkezelés](#1.4.-Fájlkezelés)
    * [1.5. Tulajdonságok](#1.5.-Tulajdonságok)
    * [1.6. Fontosabb parancsok](#1.6.-Fontosabb-parancsok)
* [2. Regex](#2.-Regex)
* [3. Irodalom](#3.-Irodalom)

## 1. Shell

### 1.1. Alapfogalmak

Kernel (rendszermag): erőforrások kezelése, absztrakció

* feladatütemezés (folyamatok hozzáférése a CPU-hoz)
* memória (RAM) kezelés
* fájlrendszer
* I/O
* stb.

Alkalmazások:

* külön programok
* a kernelen keresztül kommunikálnak másokkal

Shell (rendszerhéj): karakteres felhasználói felület alkalmazások futtatására

* maga is alkalmazás, több féle van, cserélhető
    * mostanában Linux-ok többségén és OS X-en a [__bash__](https://www.gnu.org/software/bash/) az alapértelmezett
* programozási nyelv (interpretált)

In [ ]:
%%bash
 ls -la

### 1.2. Történelem

* 1969: UNIX oprendszer. Kezdetben hozzávágják mindenkihez, aki gépet vesz, aztán rájönnek, hogy a szoftver is lehet üzlet $\to$ zárt lesz.
* 1983: GNU (GNU's Not Unix). UNIX-szerű, de nem tartalmaz UNIX-ból származó kódot. Open source. Kernel nincs (Hurd), minden más van (C fordító, debugger és library, shell, make, TeX, ablak kezelő, stb.).
* 1991: Linux kernel.
* Azóta:
    * __GNU/Linux__. Linux kernel + GNU eszközök + mindenféle más. Röviden ezt nevezik Linux-nak.
    * __UNIX-like__ vagy __\*NIX__: olyan rendszerek, amik _hasonlóan néznek ki_, mint a UNIX (GNU/Linux, BSD, OS X, stb.)

### 1.3. Hogy működik?

**Keresési sorrend**

* _alias_-ok között (`alias <név>='<utasítások>'`, pl. `alias ll='ls -l'`)
* _builtin_-ek között (pl. `alias`, `type`, `echo`, bővebben l. [itt](https://www.gnu.org/software/bash/manual/html_node/Bash-Builtins.html))
* *alkalmazás*ok között (pl. `ls`, `grep`, stb.)

Mi számít külső alkalmazásnak? Ami a `$PATH` változóban felsorolt könyvtárakban van.

In [ ]:
%%bash

echo $PATH

**Változók**

* minden string
* a szóköz delimiter, utána új kifejezés kezdődik, ezért
    * értékadásban az `=` körül nincs szóköz
    * ha az érték szóközt tartalmaz, akkor idézőjel kell
* duplaidézőjelnél van helyettesítés, szimplánál nincs
* hivatkozás `$<változó>` vagy `${<változó>}` formában

In [ ]:
%%bash

# értékadás
a=alma #barack
b="$a barack"
c='$a citrom'

# hivatkozás
echo $a
echo $b
echo $c

**Alkalmazás indítása**

Ha külső parancsot kell végrehajtania, a shell

1. létrehozza saját maga klónját (fork) -- gyerekfolyamat
2. a forkban futtatja a kapott parancsot (exec).

$\to$ Semmilyen változás nem terjed visszafelé, minden csak a gyerekfolyamatokban érvényes (pl. PATH megváltoztatása, új alias, stb). Ez a shell szkriptekre is igaz: azok is új shell-ben futnak, nincs hatásuk az indíto shell-re.

In [ ]:
%%bash
echo 'eredeti:' $valtozo
bash
valtozo='érték'
echo 'uj:' $valtozo
exit
echo 'eredeti:' $valtozo

### 1.4. Fájlkezelés

Alapvető parancsok

* `ls`: könyvtár tartalmának listázása
* `mkdir <könyvtár>`: könyvtár létrehozása
* `rmdir <könyvtár>`: könyvtár törlése
* `cp <honnan> <hova>`: fájl/könyvtár másolása
* `mv <mit> <mire>`: fájl/könyvtár átnevezése/mozgatása 
* `rm <fájl>`: törlés
* `pwd`: aktuális könyvtár

Kiemelt könyvárak rövidítései:

* `.`: jelenlegi könyvtár
* `..`: szülő könyvtár
* `/`: gyökér könyvtár

A `..` segítségével lehet relatív elérési utat megadni visszafelé is.

### 1.5. Tulajdonságok

**Programok indítása, paraméterezése**

paraméterek:

* opcionálisak vagy kötelezőek
* rövidítettek vagy hosszúak (pl. `-a` vagy `--all`)
* értéktelenek vagy értékesek esetleg opcionálisan értékesek (pl. `--color` vagy `--color=never`).

In [ ]:
%%bash
ls
echo
ls --all -l
echo
ls -la
echo
ls -la --color=always

**Saját programok futtatása**

* [shebang](https://en.wikipedia.org/wiki/Shebang_%28Unix%29)

In [ ]:
%%bash
cat hello.sh

In [ ]:
%%bash
bash hello.sh

In [ ]:
%%bash
./hello.sh

**Jogosultságkezelés**

Jogok:

* olvasási (**r**)
* írási (**w**)
* végrehajtási (**x**)

Felhasználók:

* Tulajdonos (**u**)
* Csoport (**g**)
* Mások (**o**)

Parancsok:

* `chmod`: jogosultság megváltoztatása
* `chown`: tulajdonos / csoport megváltoztatása

In [ ]:
%%bash
chmod u+x hello.sh
./hello.sh
chmod u-x hello.sh # visszaállítás

In [ ]:
%%bash

# futtatás máshonnan
cd ..
./03.ShellRegex/hello.sh
cd  03.ShellRegex

**Parancsbehelyettesítés**

`` `<parancs>` `` vagy `$(<parancs>)` formában

In [ ]:
%%bash

# futtatás abszolút elérési úttal
#pwd
/`pwd`/hello.sh

**Wildcard-ok**

In [ ]:
%%bash
ls ?hell*

**Pipe**

A`|` karakterrel lehet egy parancs kimenetét átadni a következő parancsnak bemenetként.

In [ ]:
%%bash

ls -a | wc -l

**Átirányítás**

Standard kommunikációs csatornak:

* stdin
* stdout
* stderr

* `command <filename`: fájl $\to$ stdin
* `command >filename`: stdout $\to$ fájl
* `command 2>filename`: stderr $\to$ fájl
* `>>`: appendálás

In [ ]:
%%bash

# stdout és stdin
ls -la >list.txt
cat <list.txt

# stderr
./hello.sh 2>err.txt
echo
cat err.txt

# takarítás
rm *.txt

**Visszatérési érték, elágazás**

* 0: jó
* nem 0: valamilyen hibakód

In [ ]:
%%bash

echo $?
./hello.sh
echo $?

Elágazás röviden:

```sh
cmd1 && cmd2 || cmd3
```

In [ ]:
%%bash

./hello.sh && echo "jól lefutott" || echo "jól nem futott le"

### 1.6. Fontosabb parancsok

## 2. Regex

Valami.

## 3. Irodalom